In [ ]:
import matplotlib.image as img  
import matplotlib.pyplot as plt

import xarray as xr
import numpy as np
import hvplot.xarray
import holoviews as hv

import panel as pn
import plotly.express as px
import pandas as pd


import os, random
%matplotlib inline

#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
def display_imgs(x):
    columns = 4
    rows = 2
    fig, axs = plt.subplots(rows,columns, figsize=(columns*8, rows*8))
    axs = axs.ravel()
    for idx, a in enumerate(axs):
          a.axis('off')
          a.imshow((x[idx]*255).astype(int), cmap='Greys_r')
    #plt.show()

    return fig

In [ ]:
def display_imgs_hv(ds, ds_pairs):
    plt_rgby = ds.hvplot.rgb(x='y', y='x', bands='channel',  aspect='equal', xaxis=False, yaxis=None)
    #plt_rgb = ds.hvplot(x='y',y='x',groupby='channel',width=400, aspect='equal', xaxis=False, yaxis=None )
    plt_pairs = ds_pairs.hvplot.rgb(x='y', y='x', bands='channel', groupby='pair',  aspect='equal', xaxis=False, yaxis=None)

    return plt_pairs.opts(axiswise=True) 

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def display_imgs_plotly(x):
    rows = 2
    cols = 4
    fig = make_subplots(rows=rows, cols=cols, start_cell="bottom-left", shared_xaxes=True)
    for idx, a in enumerate(range(cols*rows)):
        col = int(idx%cols + 1)
        row = int(np.floor(idx/cols) + 1)
        if idx < cols: cmap = 'gray_r'
        else: cmap = 'Viridis'
        fig.add_trace(px.imshow((x[idx])).data[0],
                    row=row, col=col)

    fig.update_layout(coloraxis_showscale=False)
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    return fig

In [ ]:
dropdown_list = ['0.  Nucleoplasm',   
                '1.  Nuclear membrane',   
                '2.  Nucleoli',   
                '3.  Nucleoli fibrillar center',   
                '4.  Nuclear speckles',   
                '5.  Nuclear bodies',   
                '6.  Endoplasmic reticulum',   
                '7.  Golgi apparatus',   
                '8.  Peroxisomes',   
                '9.  Endosomes',   
                '10.  Lysosomes',   
                '11.  Intermediate filaments',   
                '12.  Actin filaments',   
                '13.  Focal adhesion sites',   
                '14.  Microtubules',   
                '15.  Microtubule ends',   
                '16.  Cytokinetic bridge',   
                '17.  Mitotic spindle',   
                '18.  Microtubule organizing center',   
                '19.  Centrosome',   
                '20.  Lipid droplets',   
                '21.  Plasma membrane',   
                '22.  Cell junctions',   
                '23.  Mitochondria',   
                '24.  Aggresome',   
                '25.  Cytosol',   
                '26.  Cytoplasmic bodies',   
                '27.  Rods & rings'] 


In [ ]:
def sel_png_plot(dir_in="../data/train/"):
    pred_protein = []
    str_list = ['_red.png', '_blue.png', '_green.png','_yellow.png']
    filename = random.choice(os.listdir(dir_in)).rpartition('_')
    
    target_val = targets.query('Id == @filename[0]').Target.str.split(' ').values[0]
    target_val = target_val[0] # just check the first answer
    targ_list.append(target_val)
    print(targ_list)

    img_array = np.array([img.imread(dir_in+filename[0]+end_str) for end_str in str_list]).transpose([1,2,0])

    ds = xr.Dataset({'image': (('x', 'y','channel'), img_array)},
        coords={'x': np.arange(0,img_array.shape[0]), 'y': np.arange(0,img_array.shape[0]),
                'channel': ['red','green','blue','yellow']})

    img_array_gr= img_array.copy()
    img_array_gr[:,:,[1,3]] = img_array[:,:,[1,3]]*0
    img_array_gb= img_array.copy()
    img_array_gb[:,:,[0,3]] = img_array[:,:,[0,3]]*0
    img_array_yb = img_array.copy()
    img_array_yb[:,:,[0,1]] = img_array[:,:,[0,1]]*0

    img_array_pairs = np.array((img_array[:,:,0:3], img_array_gr[:,:,0:3], 
                                img_array_gb[:,:,0:3], img_array_yb[:,:,1:4]))
    ds_pairs = xr.Dataset({'image': (('pair','x', 'y','channel'), img_array_pairs)},
        coords={'x': np.arange(0,img_array.shape[0]), 'y': np.arange(0,img_array.shape[0]),
                'channel': ['red','green','blue'], 
                'pair':['rgb', 'rg', 'gb', 'yb'] })

    all_img = [img_array[:,:,0], img_array[:,:,1], img_array[:,:,2], img_array[:,:,3],
            img_array[:,:,0:3], img_array_gr[:,:,0:3], img_array_gb[:,:,0:3], img_array_yb[:,:,1:4]]

    return display_imgs(all_img)
#    return display_imgs_plotly(all_img)
#    return display_imgs_hv(ds, ds_pairs)
     



In [ ]:
hv.extension('bokeh')
fig_container = pn.pane.Matplotlib(sel_png_plot())

targ_list = []
targets = pd.read_csv('../data/train.csv')

text_list = []
button = pn.widgets.Button(name="Click me", button_type="primary")
def new_image(event):
    fig_container.loading = True
    fig = sel_png_plot()
    fig_container.object = fig
    fig_container.loading = False

def save_text(event):
    text_list.append(text_input.value)
    new_image(event)

def calc_acc(event):
    #targ_list = targ_list[:-1] # last one isn't used
    if len(targ_list[:-1]) == len(text_list):
        acc_out = sum(1 for x,y in zip(text_list,targ_list[:-1]) if x == y) / len(targ_list)
        print('Accuracy for ' + str(len(targ_list)) +': '+ acc_out)
    else:
        print("Error, lists don't line up, try again")
text_input = pn.widgets.Select(name='Select protein', options=dropdown_list)
button2 = pn.widgets.Button(name="Save selection", button_type="primary")
button2.on_click(save_text)

#forward = pn.widgets.Button(name='\u25b6', width=50)
#forward.on_click(new_image)

button3 = pn.widgets.Button(name="Calculate accuracy", button_type="primary")
button3.on_click(calc_acc)


sel_box = pn.WidgetBox('Selection', text_input, button2, button3)
img_box = pn.WidgetBox('Image', fig_container)
pn.Column(sel_box, img_box).show()
#pn.Row(pn.Column(fig_container, forward), pn.Column(text_input, button2), width=200).show()

### compare output

In [ ]:
len(targ_list[:-1]), len(text_list)

In [ ]:
pn.extension()
import matplotlib.pyplot as plt
import numpy as np
import panel as pn
from matplotlib.backends.backend_agg import \
    FigureCanvas  # not needed for mpl >= 3.1

pn.extension(sizing_mode="stretch_width")

def make_fig():
    fig, _ = plt.subplots()
    plt.hist(np.random.random(10))
    plt.close()
    FigureCanvas(fig) # not needed for mpl >= 3.1
    return fig

fig_container = pn.pane.Matplotlib(make_fig())

button = pn.widgets.Button(name="Click me", button_type="primary")
def b(event):
    fig_container.loading = True
    fig = make_fig()
    fig_container.object = fig
    fig_container.loading = False

button = pn.widgets.Button(name="Click me", button_type="primary")
button.on_click(b)

app = pn.template.MaterialTemplate(
    site="Awesome Panel",
    title="Updating Matplotlib",
    sidebar = [button],
    main = [pn.Card(fig_container)],
)

app.servable()

In [ ]:
pn.extension()
text_input = pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...')
#slider = pn.widgets.FloatSlider(name='Another widget', width=200)
forward = pn.widgets.Button(name='\u25b6', width=50)
forward.on_click(sel_png_plot()) # on click run function
#print(forward.clicks) # num clicks
pn.Column(text_input, forward, width=200)

In [ ]:
!! load csv too, compare typed and target, third button to calculate

In [ ]:
text_list

In [ ]:
import panel as pn
hv.extension('bokeh')

pane = pn.panel(sel_png_plot())
#pane.object = sel_png_plot()
text_input = pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...')
button = pn.widgets.Button(name="Click me", button_type="primary")
button.on_click(sel_png_plot())

pn.Column(text_input, pane, button, width=200)

In [ ]:
import panel as pn
pn.extension()
output_list = []
# Create and organize basic elements to show where things go
#file_input = pn.widgets.FileInput()
continent = pn.widgets.Select(
    value='Asia', 
    options=['Africa', 'Asia', 'Europe']
)
item_selector = pn.widgets.MultiSelect()
controls = pn.Column(item_selector, "")
layout = pn.Row(item_selector, pn.Column(""))
#output_list.append(continent.value)

# Set up selector object
@pn.depends(file_contents=item_selector, watch=True)
def _item_selector(file_contents):
    # Dummy file contents
    items = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'] 
    item_selector.options = items
    item_selector.value = ['a', 'c']
    item_selector.size = 12
    item_selector.width = 120
    layout[0][1] = item_selector

# Define interactivity based on selector object
@pn.depends(selected_items=item_selector, watch=True)
def _col(selected_items):
    col = pn.Column()
    for item in selected_items:
        col.append(item)
    layout[1] = col

layout

In [ ]:
output_list

In [ ]:
pane.object = burglaries

In [ ]:
sel_png_plot()

In [ ]:

pn.extension()

pn.interact(sel_png_plot)

In [ ]:
text_input = pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...')
text_input

In [ ]:
text_input[0]

In [ ]:
from dash import Dash, dcc, html, Input, Output, State

fig = sel_png_plot()

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
    html.Div(dcc.Input(id='input-on-submit', type='text')),
    html.Button('Submit', id='submit-val', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a value and press submit')
    ]),
    html.Div(
      children=[
      dcc.Graph(figure=fig)])

])


@app.callback(
    Output('container-button-basic', 'children'),
    Input('submit-val', 'n_clicks'),
    State('input-on-submit', 'value')
)
def update_output(n_clicks, value):
    return 'The input value was "{}" and the button has been clicked {} times'.format(
        value,
        n_clicks
    )


if __name__ == '__main__':
    app.run_server()